文本相似度计算 TF-IDF
http://dblab.xmu.edu.cn/blog/1766-2/
Spark 2.1.0 入门：特征抽取 — TF-IDF(Python版)


tf-idf
https://baike.baidu.com/link?url=vSHemecmjETsta8JO6IPSN_mDMQdV4d0mIB7KCLrB-XWEq28oggFiBMv0gJQn93jiXMVOC9NAFgtdJ56O5el8a

In [1]:
import os
import sys
spark_home = os.environ.get('SPARK_HOME',None)
print(spark_home)
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.10.4-src.zip'))

exec(open(os.path.join(spark_home,'python/pyspark/shell.py')).read())

/home/yang0/installer/spark-2.4.3-bin-hadoop2.7
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.6.5 (default, Jul 11 2019 14:32:02)
SparkSession available as 'spark'.


In [2]:
from pyspark.ml.feature import HashingTF,IDF,Tokenizer

sentenceData = spark.createDataFrame([(0, "I heard about Spark and I love Spark"),(0, "I wish Java could use case classes"),(1, "Logistic regression models are neat")]).toDF("label", "sentence")

#用tokenizer对句子进行分词
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
print(tokenizer)
wordsData = tokenizer.transform(sentenceData)

print(wordsData.collect())

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData) #分词序列被变换成一个稀疏特征向量

# 使用IDF来对单纯的词频特征向量进行修正，使其更能体现不同词汇对文本的区别能力，
# IDF是一个Estimator，调用fit()方法并将词频向量传入，即产生一个IDFModel
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "features").show()

Tokenizer_e44ba1b14916
[Row(label=0, sentence='I heard about Spark and I love Spark', words=['i', 'heard', 'about', 'spark', 'and', 'i', 'love', 'spark']), Row(label=0, sentence='I wish Java could use case classes', words=['i', 'wish', 'java', 'could', 'use', 'case', 'classes']), Row(label=1, sentence='Logistic regression models are neat', words=['logistic', 'regression', 'models', 'are', 'neat'])]
+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(20,[0,5,9,13,17]...|
|    0|(20,[2,7,9,13,15]...|
|    1|(20,[4,6,13,15,18...|
+-----+--------------------+



官方文档
https://spark.apache.org/docs/latest/ml-features.html#tf-idf

Pyspark Word2Vec + jieba 训练词向量流程
https://zhuanlan.zhihu.com/p/60532089

Spark MLlib实现的中文文本分类–Naive Bayes
http://lxw1234.com/archives/2016/01/605.htm

于Spark的TF-IDF算法的中文文本相似度实现
https://blog.csdn.net/cap3396g/article/details/79256625

使用spark计算文档相似度
https://blog.csdn.net/weixin_34413103/article/details/92699339

**中文分词**


In [99]:
import jieba

DICT_LIST = ["./jieba_dict/medicine.dict", "./jieba_dict/user.dict"]
STOP_WORDS = "./jieba_dict/stopwords.txt"

stopwordset = set()

# 加载用户字典
def loadDict():
    loadStopWords()
    loadUserDicts()


def loadUserDicts():
    for dict in DICT_LIST:
        jieba.load_userdict(dict)

def loadUserDict():
    jieba.load_userdict(USER_DICT)

#定义停词
def loadStopWords():
    global stopwordset
    with open(STOP_WORDS, 'r', encoding='utf-8') as sw:
        stopwordset = sw.readlines()
        stopwordset = [x.strip() for x in stopwordset]

def cutWords(s, cutAll=False):
    loadDict()
    words = jieba.cut(s, cut_all=cutAll)
    words = [w for w in words if w not in stopwordset]
    return words
#     return ' '.join(words)

print(cutWords("你好，这是医保板蓝根"))

['这是', '医保', '板蓝根']


In [100]:
import pandas as pd

from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, ArrayType

# rdd方式的读取
# 第一次toDF要先推导字段类型，第二次再指定字段名称
cnData = sc.wholeTextFiles("./cn_data/").toDF().toDF("file","text")
cnData.printSchema()

cnData = cnData.withColumn('words',cutWords_udf(cnData['text']))
cnData.first()

#计算词频
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
featurizedData = hashingTF.transform(cnData) #分词序列被变换成一个稀疏特征向量

#计算TF-IDF值
# 如果有一个词语只在a文档出现，在其他文档出现的次数少，那么该词的idf值就高，适合分类
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

rescaledData = idfModel.transform(featurizedData)
rescaledData.select("file", "features").count()

rows = rescaledData.select("file", "features").collect()

root
 |-- file: string (nullable = true)
 |-- text: string (nullable = true)



In [87]:

for row in rows:
    print(row.file)

file:/mnt/e/testProject/sparkTest/cn_data/1.txt
file:/mnt/e/testProject/sparkTest/cn_data/2.txt
file:/mnt/e/testProject/sparkTest/cn_data/3.txt
file:/mnt/e/testProject/sparkTest/cn_data/4.txt


In [89]:
print(rows[0].features.squared_distance(rows[1].features))
print(rows[0].features.squared_distance(rows[2].features))
print(rows[0].features.squared_distance(rows[3].features))
print(rows[1].features.squared_distance(rows[2].features))


3691.1335557440075
1181.8652551579291
1530.2595263280284
2793.8133277847924


In [101]:
# 余弦相似度
from scipy.spatial.distance import cosine

print(cosine(rows[0].features.toArray(), rows[1].features.toArray()))
print(cosine(rows[0].features.toArray(), rows[2].features.toArray()))
print(cosine(rows[0].features.toArray(), rows[3].features.toArray()))
print(cosine(rows[1].features.toArray(), rows[3].features.toArray()))

0.980447656477595
0.9909107174983364
0.9895697103885169
0.9819028177096234
